In [1]:
# -*- coding: utf-8 -*-
"""
Export risk feature classes to CSV with per-field aliases and numeric transforms.

For each FC in <gdb>\risk:
  - Map FC to a risk column in sheet 'Поля для дашборда'
  - Keep rows where that risk column is non-empty; the cell is the TARGET ALIAS
  - Read only those fields from the FC (no geometry)
  - Per field:
      * If Excel 'комментарий' cell is non-empty -> multiply by 100
      * If field is iOverTot or iOverLive -> value = value*100 - 100
      * Round to N decimals from Excel column 'количество знаков после запятой'
  - Rename to alias and export CSV

Requires: arcpy, pandas, openpyxl
"""

import os
import re
import sys
import time
from pathlib import Path

import arcpy
import pandas as pd

# ========= CONFIGURE =========
region     = "Yakutia"
gdb_path   = r"C:\Users\User\Documents\ЦГД\Yakutia_RiskProfile_30-10-2025\Yakutia_RiskProfile_30-10-2025\Yakutia_RiskProfile\Default.gdb"
RISKS_FDS  = "risk"
EXCEL_PATH = r"C:\Users\User\Documents\ЦГД\Структура базы_v03.xlsx"   # or the updated "(1).xlsx"
SHEET_NAME = "Поля для дашборда"
OUT_DIR    = os.path.join(os.path.dirname(gdb_path), "risk_csv")
ENCODING   = "utf-8-sig"

# Exact column headers (with robust fallbacks)
FIELDNAME_HEADER   = "Поле шейпа (10 симв., латынь)"
DECIMALS_HEADER    = "количество знаков после запятой"
COMMENT_HEADER     = "комментарий"

# ========= HELPERS =========
def print_flush(*args, **kwargs):
    print(*args, **kwargs); sys.stdout.flush()

def secs(dt): return f"{dt:.2f}s"

def normalize(s: str) -> str:
    return re.sub(r"[_\W]+", "", str(s).strip().lower(), flags=re.UNICODE)

def is_truthy_cell(v) -> bool:
    if pd.isna(v): return False
    if isinstance(v, (int, float)):
        try: return float(v) != 0.0
        except Exception: return True
    s = str(v).strip()
    return len(s) > 0 and s not in {"0", "нет", "no", "false", "False", "-"}

def find_best_risk_column(fc_name: str, candidate_cols):
    """Exact (ci) → normalized equality → substring overlap."""
    fc_l = fc_name.strip()
    fc_n = normalize(fc_l)

    for c in candidate_cols:
        if fc_l.lower() == str(c).strip().lower():
            return c
    for c in candidate_cols:
        if fc_n == normalize(c):
            return c
    best, best_len = None, 0
    for c in candidate_cols:
        cn = normalize(c)
        if fc_n in cn or cn in fc_n:
            L = min(len(fc_n), len(cn))
            if L > best_len:
                best, best_len = c, L
    return best

def featureclass_to_df(fc_path: str, fields: list) -> pd.DataFrame:
    """Use SearchCursor so NULLs stay as None (avoids NumPy dtype issues)."""
    if not fields:
        return pd.DataFrame()
    rows = []
    with arcpy.da.SearchCursor(fc_path, fields) as cur:
        for row in cur:
            rows.append(row)
    if not rows:
        return pd.DataFrame(columns=fields)
    return pd.DataFrame.from_records(rows, columns=fields)

# ========= READ EXCEL =========
t0 = time.time()
print_flush(f"[1/4] Reading Excel: {EXCEL_PATH} (sheet='{SHEET_NAME}') ...")
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
print_flush(f"       Rows: {len(df)}; Columns: {list(df.columns)}")

# Robust header resolution
cols_lower = {str(c).strip().lower(): c for c in df.columns}

def pick(colname, *fallbacks):
    c = cols_lower.get(colname.strip().lower())
    if c: return c
    for fb in fallbacks:
        cc = cols_lower.get(fb.strip().lower())
        if cc: return cc
    raise ValueError(f"Не найден столбец: '{colname}' (с учетом возможных вариантов: {fallbacks})")

field_col    = cols_lower.get(FIELDNAME_HEADER.lower()) or pick(FIELDNAME_HEADER, "название поля", "field_name_tot", "имя поля")
decimals_col = cols_lower.get(DECIMALS_HEADER.lower())  or pick(DECIMALS_HEADER, "знаков после запятой", "decimal places")
comment_col  = cols_lower.get(COMMENT_HEADER.lower())   or pick(COMMENT_HEADER, "коммент", "comment")

# Normalize core columns
df[field_col]    = df[field_col].astype(str).str.strip()
# decimals may be numeric or text; keep as-is for now
# comment stays as-is; presence indicates x100

# Risk columns are *everything else* (each cell holds the ALIAS to use)
meta_cols = {field_col, decimals_col, comment_col}
risk_cols_in_sheet = [c for c in df.columns if c not in meta_cols]
if not risk_cols_in_sheet:
    raise RuntimeError("В листе не обнаружены столбцы с рисками (кроме полей, знаков и комментариев).")
print_flush(f"       Risk columns detected: {len(risk_cols_in_sheet)}")

# ========= SCAN GDB / EXPORT =========
arcpy.env.workspace = os.path.join(gdb_path, RISKS_FDS)
fcs = arcpy.ListFeatureClasses() or []
print_flush(f"\n[2/4] Feature dataset '{RISKS_FDS}' -> {len(fcs)} feature classes found")

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

exported = 0
for idx, fc in enumerate(fcs, start=1):
    if "template" in fc.lower():
        print_flush(f"[FC {idx}/{len(fcs)}] {fc} -> skipped (template)")
        continue

    fc_path = os.path.join(gdb_path, RISKS_FDS, fc)
    print_flush(f"\n[FC {idx}/{len(fcs)}] {fc} -> {fc_path}")

    # Find the matching risk column (holds target aliases directly)
    risk_col = find_best_risk_column(fc, risk_cols_in_sheet)
    if not risk_col:
        print_flush("       ⚠ No matching risk column in Excel; skipping.")
        continue
    print_flush(f"       Matched risk column in Excel: '{risk_col}' (alias source)")

    # Select rows where the alias cell (this risk column) is non-empty
    mask = df[risk_col].apply(is_truthy_cell)
    sub = df.loc[mask, [field_col, decimals_col, comment_col, risk_col]].copy()
    if sub.empty:
        print_flush("       (No fields marked/aliased for this risk) → skip")
        continue

    # Prepare per-field settings
    sub.rename(columns={
        field_col: "field_name",
        decimals_col: "decimals",
        comment_col: "comment",
        risk_col: "alias"
    }, inplace=True)

    # Clean up decimals (default to 2 if empty/invalid)
    def parse_dec(v):
        if pd.isna(v): 
            return 2
        try:
            n = int(float(str(v).replace(",", ".")))
            return max(0, min(10, n))  # clamp to a reasonable range
        except Exception:
            return 2
    sub["decimals"] = sub["decimals"].apply(parse_dec)

    # Determine which fields exist in the FC
    try:
        fields = arcpy.ListFields(fc_path)
    except Exception as e:
        print_flush(f"       ✖ ERROR ListFields: {e}")
        continue

    data_fields = [f for f in fields if f.type not in ("OID", "Geometry")]
    present = {f.name for f in data_fields}

    # Keep only fields present in the FC
    sub = sub[sub["field_name"].isin(present)].drop_duplicates("field_name")
    if sub.empty:
        print_flush("       (Marked fields not present in this FC) → skip")
        continue

    # Read only those fields
    selected_fields = list(sub["field_name"])
    df_fc = featureclass_to_df(fc_path, selected_fields)

    # Apply transforms per field, then rename to alias
    # Build lookup dicts for speed
    dec_map   = dict(zip(sub["field_name"], sub["decimals"]))
    print(dec_map)
    comm_map  = dict(zip(sub["field_name"], sub["comment"]))
    alias_map = dict(zip(sub["field_name"], sub["alias"]))

    SPECIAL_FIELDS = {"iOverTot", "iOverLive"}

    for fld in selected_fields:
        series = df_fc[fld]
        s_num = pd.to_numeric(series, errors="coerce")
        comment_text = str(comm_map.get(fld, "")).strip().lower()
        nd = int(dec_map.get(fld, 2))  # per-field decimals

        applied = False

        # --- 1) 'gut' → no scaling/special, but DO round (and int if 0 d.p.) ---
        if "gut" in comment_text:
            if s_num.notna().any():
                s_num = s_num.round(nd)
                if nd == 0:
                    try:
                        s_num = s_num.astype("Int64")  # nullable int keeps NaN
                    except Exception:
                        s_num = s_num.astype(float).astype("Int64", errors="ignore")
                df_fc[fld] = s_num
            # non-numeric stays untouched (but still renamed below)

        else:
            # --- 2) Special rule (highest priority) ---
            if fld in SPECIAL_FIELDS and s_num.notna().any():
                s_num = s_num * 100.0 - 100.0
                applied = True

            # --- 3) комментарий scaling (only if not special) ---
            elif is_truthy_cell(comment_text) and s_num.notna().any():
                clean = comment_text.replace(" ", "")
                if "/1000000" in clean:
                    s_num = s_num / 1_000_000.0
                elif "%" in clean or clean:
                    s_num = s_num * 100.0
                applied = True

            # --- 4) Round everything numeric; int if 0 d.p. ---
            if applied or s_num.notna().any():
                s_num = s_num.round(nd)
                if nd == 0:
                    try:
                        s_num = s_num.astype("Int64")
                    except Exception:
                        s_num = s_num.astype(float).astype("Int64", errors="ignore")
                df_fc[fld] = s_num

            # # --- 6) Post-correction: ensure sLiveFrac >= sTotFrac ---
            # if "sLiveFrac" in df_fc.columns and "sTotFrac" in df_fc.columns:
            #     try:
            #         # Work only on numeric values
            #         s_live = pd.to_numeric(df_fc["sLiveFrac"], errors="coerce")
            #         s_tot  = pd.to_numeric(df_fc["sTotFrac"],  errors="coerce")

            #         # Replace where live < total
            #         mask = s_live < s_tot
            #         if mask.any():
            #             df_fc.loc[mask, "sLiveFrac"] = s_tot[mask]
            #             print_flush(f"       Adjusted {mask.sum()} row(s): sLiveFrac < sTotFrac → set equal.")
            #     except Exception as e:
            #         print_flush(f"       ⚠ Error during sLiveFrac/sTotFrac correction: {e}")

        # --- 5) Rename to alias (always) ---
        alias = str(alias_map.get(fld, fld)).strip()
        if alias:
            df_fc.rename(columns={fld: alias}, inplace=True)
            

    # Export CSV
    out_name = f"{region}_{fc}.csv"
    out_path = os.path.join(OUT_DIR, out_name)
    df_fc.to_csv(out_path, index=False, encoding=ENCODING, sep = ';')
    exported += 1
    print_flush(f"       ✔ Exported {df_fc.shape[0]} rows, {df_fc.shape[1]} cols -> {out_name}")

# ========= SUMMARY =========
print_flush(f"\n[3/4] Exported CSVs: {exported}")
print_flush(f"[4/4] Output folder: {OUT_DIR}")


[1/4] Reading Excel: C:\Users\User\Documents\ЦГД\Структура базы_v03.xlsx (sheet='Поля для дашборда') ...
       Rows: 112; Columns: ['Группа полей', 'Поле шейпа (10 симв., латынь)', 'Псевдоним (понятный, русский)', 'Размерность исходных данных', 'Псевдоним в дашборде', 'Wildfire', 'Cold', 'Heat', 'Rain', 'Wind', 'Freez', 'Hail', 'Thunderstorm', 'Drought', 'Flood', 'Permafrost', 'количество знаков после запятой', 'комментарий']
       Risk columns detected: 15

[2/4] Feature dataset 'risk' -> 12 feature classes found
[FC 1/12] Yakutia_hazardTemplate_MO -> skipped (template)

[FC 2/12] Yakutia_hazHeat_MO -> C:\Users\User\Documents\ЦГД\Yakutia_RiskProfile_30-10-2025\Yakutia_RiskProfile_30-10-2025\Yakutia_RiskProfile\Default.gdb\risk\Yakutia_hazHeat_MO
       Matched risk column in Excel: 'Heat' (alias source)
{'MO': 2, 'areaLiveF': 0, 'sTot': 0, 'sTotFrac': 0, 'sLive': 0, 'sLiveFrac': 0, 'spTot': 0, 'spTotFrac': 0, 'spLive': 0, 'spLiveFrac': 0, 'ieMeanTot': 1, 'ieMeanLive': 1, 'ieMaxTot':